In [1]:
from utils import *

In [2]:
struct_df_file = {
    'cec_m1' : 'a_cecal_mouse_1.hdf5',
    'cec_m2' : 'a_cecal_mouse_2.hdf5',
    'ser_m1' : 'a_serum_mouse_1.hdf5',
    'ser_m2' : 'a_serum_mouse_2.hdf5'
}

In [3]:
for dfn in ['cec_m1', 'cec_m2', 'ser_m1', 'ser_m2']:
    df1 = pd.read_hdf(f'generated/{struct_df_file[dfn]}', f'rep_0')
    df2 = pd.read_hdf(f'generated/{struct_df_file[dfn]}', f'rep_1')
    df3 = pd.read_hdf(f'generated/{struct_df_file[dfn]}', f'rep_2')
    meandf = pd.DataFrame(columns=df1.keys())
    meandf['Metabolite Name'] = df1['Metabolite Name']
    meandf = meandf.fillna(0)
    stddf = pd.DataFrame(columns=df1.keys())
    stddf['Metabolite Name'] = df1['Metabolite Name']
    stddf = stddf.fillna(0)
    for k in df1.keys():
        if k == 'Metabolite Name':
            continue
        # df1 = df1.reset_index(drop=True)
        # df2 = df2.reset_index(drop=True)
        # df3 = df3.reset_index(drop=True)
        
        mask1 = (df1[k] != 0)
        mask2 = (df2[k] != 0)
        mask3 = (df3[k] != 0)

        maskall3 = mask1 & mask2 & mask3
        maskall2 = (mask1 & mask2 & ~mask3) | (mask1 & ~mask2 & mask3) | (~mask1 & mask2 & mask3)
    
        # meandf.loc[mask1 & mask2 & mask3, k] = (df1[mask1 & mask2 & mask3][k] + df2[mask1 & mask2 & mask3][k] + df3[mask1 & mask2 & mask3][k]) / 3
        # meandf.loc[maskatl2, k] = (df1[k][maskatl2] + df2[k][maskatl2] + df3[k][maskatl2]) / 2

        meandf.loc[maskall3, k] = pd.DataFrame({'df1' : df1[maskall3][k], 'df2' : df2[maskall3][k], 'df3' : df3[maskall3][k]}).mean(axis=1)
        meandf.loc[maskall2, k] = pd.DataFrame({'df1' : df1[maskall2][k], 'df2' : df2[maskall2][k], 'df3' : df3[maskall2][k]}).mean(axis=1)
        meandf.loc[((~mask1 & ~mask2 & mask3) | (mask1 & ~mask2 & ~mask3) | (~mask1 & mask2 & ~mask3)), k] = np.nan


        stddf.loc[maskall3, k] = pd.DataFrame({'df1' : df1[maskall3][k], 'df2' : df2[maskall3][k], 'df3' : df3[maskall3][k]}).std(axis=1)
        stddf.loc[maskall2, k] = pd.DataFrame({'df1' : df1[maskall2][k], 'df2' : df2[maskall2][k], 'df3' : df3[maskall2][k]}).std(axis=1)
        stddf.loc[((~mask1 & ~mask2 & mask3) | (mask1 & ~mask2 & ~mask3) | (~mask1 & mask2 & ~mask3)), k] = np.nan
    
        # stddf.loc[mask1 & mask2 & mask3, k] = np.std(df1[mask1 & mask2 & mask3][k] + df2[mask1 & mask2 & mask3][k] + df3[mask1 & mask2 & mask3][k])
        # stddf.loc[maskatl2, k] = np.std(df1[(mask1 & mask2 & ~mask3) | (
        #     mask1 & ~mask2 & mask3) ][k] + df2[(
        #     mask1 & mask2 & ~mask3) | (~mask1 & mask2 & mask3)][k] + df3[(
        #     mask1 & ~mask2 & mask3) | (~mask1 & mask2 & mask3)][k])
        # stddf.loc[((~mask1 & ~mask2 & mask3) | (mask1 & ~mask2 & ~mask3) | (~mask1 & mask2 & ~mask3)), k] = np.nan
    meandf = meandf.dropna()
    stddf = stddf.dropna()
    
    meandf.to_hdf(f'generated/{struct_df_file[dfn]}', 'mean')
    stddf.to_hdf(f'generated/{struct_df_file[dfn]}', 'std')

In [4]:
for dfn in ['cec_m1', 'cec_m2', 'ser_m1', 'ser_m2']:
    meandf = pd.read_hdf(f'generated/{struct_df_file[dfn]}', 'mean')
    stddf = pd.read_hdf(f'generated/{struct_df_file[dfn]}', 'std')

    cvdf = meandf.copy()
    notname = list(set(list(meandf.keys())) - set(['Metabolite Name']))
    cvdf[notname] = stddf[notname]/meandf[notname]

    cv15mask = cvdf[notname].iloc[:, 1:].lt(1.5).all(axis=1)

    cvfiltered = meandf[cv15mask]
    print(dfn, len(cvfiltered)*100/len(meandf))

    cvfiltered.to_hdf(f'generated/{struct_df_file[dfn]}', 'cvfiltered')

cec_m1 74.44527549239591
cec_m2 71.16733466933867
ser_m1 72.5576983876067
ser_m2 72.85617825793383
